**Benefits od paralllel:**

-- having new datasets we can easly imporve the model


In [15]:
import numpy as np
import csv
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, Normalizer
import cvxpy as cp

import preprocessing as pp


In [5]:
pd.__version__

'0.24.2'

Load datasets

In [6]:
# cleveland = pd.read_csv("data/processed.cleveland.data", header=None)
# hungarian = pd.read_csv("data/processed.hungarian.data", header=None)
# switzerland = pd.read_csv("data/processed.switzerland.data", header=None)
# va = pd.read_csv("data/processed.va.data", header=None)

In [7]:
# columns = ["Age", "Sex", "ChPain", "RestBlPrs", "SerChol", "FBlSug", "RestECG", "MaxHR", "Eia", "STdep", "PeakExST", "MajVess", "Thal", "HeartDis"]

In [8]:
# cleveland.columns = columns
# hungarian.columns = columns
# switzerland.columns = columns
# va.columns = columns

In [9]:
# def check_missing_values(df, columns):
#     indx = []
#     for col in columns:
#         tmp = df[df[col] == "?"][col]
#         indx.extend(tmp.index)
# #         print(col, tmp.count())
#     return indx

In [10]:
# idx = check_missing_values(cleveland, columns)
# cleveland.iloc[idx]

# # remove rows with missing data from cleveland dataset
# data = cleveland.loc[~cleveland.index.isin(idx)]

# #TODO: change most of the colmns to integer type
# data = data.apply(pd.to_numeric) 
# # data.dtypes

# del cleveland

# y = data["HeartDis"]

# label_mapping = {0:-1, 1:1, 2:1, 3:1, 4:1}
# y = [label_mapping[label] for label in y]

# X = data.loc[:, data.columns != 'HeartDis']

# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=123)
# del X, y, data

In [17]:
all_dtypes = pp.CollectDataTypes().get_dict_all_dtypes(label='int16')
subset_df = pd.read_csv('all_data.csv', dtype=all_dtypes, index_col=0)

numerical_cols = subset_df.select_dtypes(exclude=['object']).columns


# TRAIN - TEST splitting
subset_df_ = subset_df.dropna(how='any')
X_train, X_test, y_train, y_test = train_test_split(subset_df_[numerical_cols], 
                                                    subset_df_['loan_status'],
                                                    test_size=0.3, random_state=42)
del X_train['loan_status']
del X_test['loan_status']

y_train = list(y_train)
for i in range(len(y_train)):
    if y_train[i] == 0:
        y_train[i] = -1
    else:
        y_train[i] = 1
        
y_test = list(y_test)
for i in range(len(y_test)):
    if y_test[i] == 0:
        y_test[i] = -1
    else:
        y_test[i] = 1
        
y_test = np.array(y_test).reshape(-1,1)

In [18]:
# scaler = StandardScaler()
# scaler.fit(X_train)

# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

normalizer_ = Normalizer()
normalizer_.fit(X_train)

X_train = normalizer_.transform(X_train)
X_test = normalizer_.transform(X_test)

In [176]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='minority')
x_train_res, y_train_res = sm.fit_sample(X_train, y_train)

# Shuffle dataset
indexes = np.arange(len(x_train_res))
np.random.shuffle(indexes)

X_train = x_train_res[indexes]
y_train = np.array(y_train_res[indexes]).reshape(-1, 1)

## Parallel Consensus-based SVM

In [126]:
class SGD():
    """ Compute x_i for Stochastic Gradient Descent with 
    fixed number of iterations"""
    
    def __init__(self, n_iter=10, rho=1):
        self.n_iter = n_iter
        self.rho = rho
 
    def fit(self, A, x, u, z):
        self.x = x.reshape(-1,1)
        z = z.reshape(-1,1)
        u = u.reshape(-1,1)
        
        for i in range(self.n_iter):
            eta = 1. / (self.rho*(i+1)) #Report it! decreasing step
            
            for j, a in enumerate(A):
                a = A[j].reshape(1, -1)
                score = a.dot(self.x)
                
                if score > -1:
                    self.x -= eta*self.rho*(self.x-z+u) + eta*a.T
                else:
                    self.x -= eta*self.rho*(self.x-z+u)
                    
#                 print(self.x)

    def get_resut(self):
        return self.x

**Divide data into blocks - splitting of variables**

In [172]:
def get_chunks(data, n):
    return [data[i:i + n] for i in range(0, len(data), n)]

In [173]:
def generate_A_mat(X, y):
    # Add one at the end of all x-data vectors
#     X = X.to_numpy()
    X = np.concatenate((X, np.ones((X.shape[0], 1))), axis = 1)
    
    # -1 * y_vector
    minus_y = np.multiply(-1, y).reshape(-1, 1)
    
    # Componentiwise multiplication
    A = np.multiply(minus_y, X)
    return A

### ADMM

In [197]:
class Node():
    
    def __init__(self, A_chunk, x, rho):
        self.A = A_chunk
        self.x = x
        self.rho = rho
    
    def compute_x_i(self, u_i, z):
        self.u = u_i
        m, n = self.A.shape

        x = cp.Variable((n, 1))
        loss = cp.sum(cp.pos(1 + self.A*x))
        reg = cp.sum_squares(self.x - z + self.u)
        rho = cp.Parameter(nonneg=True)
        prob = cp.Problem(cp.Minimize(loss + rho/2*reg))
        rho.value = self.rho
        prob.solve(verbose=True)
        self.x = x.value
        del prob
        print(self.x[0])

In [198]:
class FusionCenter():
    
    def __init__(self, N, rho, lamb, nodes):
        self.N = N
        self.rho = rho
        self.lamb = lamb
        self.nodes = nodes #to observe all Node instances
        
    def _average_x_u(self):
        x = np.array(self.x_list) 
        u = np.array(self.u_list)
        
        self.x_bar = np.average(x, axis=0).reshape(-1,1)
        self.u_bar = np.average(u, axis=0).reshape(-1,1)
        
    def collect_x_u(self):
        self.x_list = []
        self.u_list = []
        
        for node in self.nodes:
            self.x_list.append(node.x)
            self.u_list.append(node.u)
        
        self._average_x_u()
    
    def compute_z(self):
        const_part = self.N * self.rho / (2*self.lamb + self.N*self.rho)
        self.z = const_part * (self.x_bar + self.u_bar)
    
    def compute_u_i(self):
        for idx, u in enumerate(self.u_list):
            self.u_list[idx] = u.reshape(-1,1) + self.x_list[idx] - self.z
    
    def check_consensus(self, error_tol):
        
        for x in self.x_list:
#             print(x - self.x_bar)
            diff = x - self.x_bar
#             print(diff)
            if (abs(diff) > error_tol).any():
#                 print(((abs(diff) > error_tol)*1).mean())
                return False
            
        # if all x_i are more-or-less the same
        return True

### def __main__:

In [199]:
# A_chunks[4].shape
# X_train.shape[0]/4

In [200]:
# Preprocess data
A = generate_A_mat(X_train, y_train)
A_chunks = get_chunks(A, 15025)

In [201]:
# Fixed parameters:
RHO = 0.0001
LAMBD = 0.000001
N = len(A_chunks)
E_TOL = 0.1

In [202]:
# First iteration:

N, m = X_train.shape
x_init = np.zeros(m+1)
z_init = np.zeros(m+1)
u_init = np.zeros(m+1)

In [203]:
node1 = Node(A_chunk = A_chunks[0], x=x_init, rho = RHO)
node2 = Node(A_chunk = A_chunks[1], x=x_init, rho = RHO)
node3 = Node(A_chunk = A_chunks[2], x=x_init, rho = RHO)
node4 = Node(A_chunk = A_chunks[3], x=x_init, rho = RHO)

node1.compute_x_i(u_i = u_init, z = z_init)
node2.compute_x_i(u_i = u_init, z = z_init)
node3.compute_x_i(u_i = u_init, z = z_init)
node4.compute_x_i(u_i = u_init, z = z_init)

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 15072, constraints m = 30050
          nnz(P) + nnz(A) = 284648
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.4414e+05   4.03e+01   4.86e+01   1.00e-01   9.23e-02s
 200   4.5392e+03   1.58e+03   7.15e-01   4.03e-06   5.89e-01s
 400   7.2118e+03   7.15e+02   8.80e-0

ValueError: OSQP solve error!

In [169]:
FC = FusionCenter(N=N, rho=RHO, lamb=LAMBD,
                 nodes = [node1, node2, node3, node4])
FC.collect_x_u()
consensus = FC.check_consensus(error_tol=E_TOL)

In [170]:
# More iterations

counter = 0 # Check the number of iterations
while not consensus:
    counter += 1
    print(counter)
    FC.compute_z()
    FC.compute_u_i()
    
    node1.compute_x_i(u_i = FC.u_list[0], z = FC.z)
    node2.compute_x_i(u_i = FC.u_list[1], z = FC.z)
    node3.compute_x_i(u_i = FC.u_list[2], z = FC.z)
    node4.compute_x_i(u_i = FC.u_list[3], z = FC.z)

    FC.collect_x_u()
    consensus = FC.check_consensus(error_tol=E_TOL)

1
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 9548, constraints m = 19002
          nnz(P) + nnz(A) = 169632
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.0962e+05   3.76e+01   5.47e+02   1.00e-01   5.63e-02s
 175   3.4898e+03   2.51e-06   7.12e-06   1.00e-01   2.40e-01s

status:               solved
solutio

 100   3.5540e+03   3.81e-06   4.37e-06   1.00e-01   1.48e-01s

status:               solved
solution polish:      unsuccessful
number of iterations: 100
optimal objective:    3553.9991
run time:             1.79e-01s
optimal rho estimate: 2.22e-02

[3.31335693e-06]
3
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 9548, constraints m = 19002
          nnz(P) + nnz(A) = 169632
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
         

 100   3.5540e+03   3.81e-06   4.37e-06   1.00e-01   1.96e-01s

status:               solved
solution polish:      unsuccessful
number of iterations: 100
optimal objective:    3553.9991
run time:             2.32e-01s
optimal rho estimate: 2.22e-02

[3.31335693e-06]
5
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 9548, constraints m = 19002
          nnz(P) + nnz(A) = 169632
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
         


status:               solved
solution polish:      unsuccessful
number of iterations: 100
optimal objective:    3553.9991
run time:             1.73e-01s
optimal rho estimate: 2.22e-02

[3.31335693e-06]
7
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 9548, constraints m = 19002
          nnz(P) + nnz(A) = 169632
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective 

 100   3.5540e+03   3.81e-06   4.37e-06   1.00e-01   1.58e-01s

status:               solved
solution polish:      unsuccessful
number of iterations: 100
optimal objective:    3553.9991
run time:             1.93e-01s
optimal rho estimate: 2.22e-02

[3.31335693e-06]
9
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 9548, constraints m = 19002
          nnz(P) + nnz(A) = 169632
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
         

ValueError: OSQP solve error!

In [147]:
for i in range(node1.x.shape[0]):
    print(node1.x[i], node2.x[i], node3.x[i])

[-3.2055704e-07] [5.0030496e-06] [9.01067686e-06]
[-9.81247911e-07] [-3.66574285e-06] [-7.85320603e-07]
[3.68123076e-07] [-2.52233179e-05] [-3.3486909e-06]
[3.45494279e-07] [6.98370372e-06] [2.71367517e-06]
[7.38877002e-07] [2.49424647e-07] [2.4347006e-05]
[-7.08578897e-07] [1.42259226e-06] [5.97897703e-05]
[1.14769431e-06] [-2.76073877e-05] [4.54125584e-05]
[8.44606812e-08] [8.87750043e-08] [6.03562248e-06]
[2.91383037e-08] [2.72911846e-06] [1.90908435e-06]
[2.31705267e-06] [0.0002952] [-0.00142905]
[0.0001923] [-5.00546447e-06] [-0.00065677]
[7.94414822e-08] [5.58188787e-06] [-5.44316932e-06]
[-1.68658983e-09] [-8.83027253e-07] [-2.21030021e-06]
[-1.01972767e-05] [-0.00021453] [0.00079122]
[-1.89061559e-05] [6.79116754e-06] [0.0003898]
[7.03138364e-08] [2.46811784e-06] [3.44226765e-06]
[-1.98738783e-07] [-1.08995658e-05] [-7.43399534e-05]
[3.69125692e-07] [5.9778496e-07] [-3.68961981e-05]
[-4.47333098e-07] [-7.89593708e-06] [7.02099964e-05]
[1.394954e-06] [-4.9589269e-05] [0.00014395

In [137]:
print("No. Iterations = ", counter)

No. Iterations =  120


**PREDICTIONS and VALIDATION**

In [138]:
w = FC.x_bar[:-1]
b = FC.x_bar[-1][0]

In [139]:
def predict(X, w, b):
    return np.sign(w.T.flatten().dot(X.T) - b)

In [140]:
y_pred = predict(X_train, w, b)

In [141]:
# accuracy
print("train_acc", accuracy_score(y_train, y_pred))
print("test_acc", accuracy_score(y_test, predict(X_test, w, b)))

train_acc 0.8016877637130801
test_acc 0.85


-------------------

In [144]:
print("train_acc", accuracy_score(y_train, sklearn_pred))
print("test_acc", accuracy_score(y_test, clf.predict(X_test)))

train_acc 0.9071729957805907
test_acc 0.85


### With Ray library for parallel computations

In [17]:
ray.init(memory=268435456, object_store_memory=134217728)

2020-01-27 23:36:40,658	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-01-27 23:36:40,660	INFO resource_spec.py:216 -- Starting Ray with 0.24 GiB memory available for workers and up to 0.12 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '192.168.1.147',
 'redis_address': '192.168.1.147:24112',
 'object_store_address': '/tmp/ray/session_2020-01-27_23-36-40_657424_4200/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-01-27_23-36-40_657424_4200/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-01-27_23-36-40_657424_4200'}

In [41]:
@ray.remote
class Node():
    
    def __init__(self, x, rho):
        self.x = x
        self.rho = rho
    
    def compute_x_i(self, A_chunk, u_i, z):
        self.u = u_i
        sgd = SGD(n_iter=10, rho=self.rho)
        sgd.fit(A_chunk, self.x, u_i, z)
        self.x = sgd.get_resut().copy()
        del sgd
        
    def read(self):
        return self.x

In [42]:
class FusionCenter():
    
    def __init__(self, N, rho, lamb):
        self.N = N
        self.rho = rho
        self.lamb = lamb
        
    def average_x_u(self, x_list, u_list=None):
        self.x_list = x_list 
        if u_list == None: 
            self.u_bar = np.average(np.array(self.u_list), axis=0).reshape(-1,1)
        else:
            self.u_list = u_list
            self.u_bar = np.average(np.array(self.u_list), axis=0).reshape(-1,1)
            self.x_bar = np.average(np.array(self.x_list), axis=0).reshape(-1,1)

    
    def compute_z(self):
        const_part = self.N * self.rho / (2*self.lamb + self.N*self.rho)
        self.z = const_part * (self.x_bar + self.u_bar)
        del const_part
    
    
    def compute_u_i(self):
        for idx, u in enumerate(self.u_list):
            self.u_list[idx] = u.reshape(-1,1) + self.x_list[idx] - self.z
    
    
    def check_consensus(self, error_tol):
        
        for x in self.x_list:
            diff = x - self.x_bar 
            if (abs(diff) > error_tol).any():
                return False
            
        # if all x_i are more-or-less the same
        return True

In [43]:
FC = FusionCenter(N=N, rho=RHO, lamb=LAMBD)
nodes = [Node.remote(x_init, RHO) for i in range(5)]

In [44]:
[node.compute_x_i.remote(A_chunk=A_chunks[i], u_i = u_init, z = z_init) for i, node in enumerate(nodes)]

[ObjectID(3981b4d170133f63a60b8762010000c801000000),
 ObjectID(beafb89ed4c765d1994fcb5c010000c801000000),
 ObjectID(9d02fc664f726154220e9109010000c801000000),
 ObjectID(92be2035553999b219702040010000c801000000),
 ObjectID(2e9b97e987e7a36051e92997010000c801000000)]

In [45]:
x_list = [node.read.remote() for node in nodes]
x_list = ray.get(x_list)

In [46]:
u_list = [u_init for i in range(5)]
FC.average_x_u(x_list, u_list)
consensus = FC.check_consensus(error_tol=E_TOL)
consensus

False

In [47]:
# More iterations
counter=0
while not consensus:
    counter += 1
    print(counter)
    
    FC.compute_z()
    FC.compute_u_i()

    [node.compute_x_i.remote(A_chunk=A_chunks[i], u_i = FC.u_list[0], z = FC.z) for i, node in enumerate(nodes)]
    
    x_list = [node.read.remote() for node in nodes]
    x_list = ray.get(x_list)

    FC.average_x_u(x_list)
    consensus = FC.check_consensus(error_tol=E_TOL)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [2]:
ray.shutdown()

### Visualisations

In [59]:
#TODO: 1)Accuracy vs. number of training samples

In [60]:
# 2)